# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,19.32,94,75,10.28,PT,1736461761
1,1,puerto natales,-51.7236,-72.4875,16.25,39,20,8.23,CL,1736461762
2,2,padangsidempuan,1.3667,99.2667,21.38,98,100,1.37,ID,1736461763
3,3,west island,-12.1568,96.8225,25.99,94,75,3.09,CC,1736461764
4,4,edinburgh of the seven seas,-37.0676,-12.3116,17.89,91,100,5.93,SH,1736461765


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    color = "City",
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 24)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,padangsidempuan,1.3667,99.2667,21.38,98,100,1.37,ID,1736461763
6,6,east london,-33.0153,27.9116,23.01,85,0,6.39,ZA,1736461767
9,9,nueva gerona,21.8833,-82.8000,20.46,59,92,8.14,CU,1736461770
12,12,blackmans bay,-43.0167,147.3167,21.52,69,37,0.45,AU,1736461773
14,14,mazamari,-11.3258,-74.5308,23.50,98,99,1.40,PE,1736461775
...,...,...,...,...,...,...,...,...,...,...
531,531,tanjung selor,2.8372,117.3642,23.32,98,79,0.49,ID,1736462337
534,534,fada n'gourma,12.0622,0.3578,22.98,18,84,4.30,BF,1736462340
565,565,tangalle,6.0240,80.7911,23.55,93,87,0.94,LK,1736462373
578,578,mabaruma,8.2000,-59.7833,22.63,85,19,1.33,GY,1736462387


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = ideal_weather[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,padangsidempuan,ID,1.3667,99.2667,98,
6,east london,ZA,-33.0153,27.9116,85,
9,nueva gerona,CU,21.8833,-82.8000,59,
12,blackmans bay,AU,-43.0167,147.3167,69,
14,mazamari,PE,-11.3258,-74.5308,98,
...,...,...,...,...,...,...
531,tanjung selor,ID,2.8372,117.3642,98,
534,fada n'gourma,BF,12.0622,0.3578,18,
565,tangalle,LK,6.0240,80.7911,93,
578,mabaruma,GY,8.2000,-59.7833,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
padangsidempuan - nearest hotel: Hotel Mega Permata
east london - nearest hotel: No hotel found
nueva gerona - nearest hotel: Hotel Bamboo
blackmans bay - nearest hotel: Villa Howden
mazamari - nearest hotel: Hospedaje El Conquistador
san fernando - nearest hotel: Pampanga Lodge and Restaurant
ocozocoautla de espinosa - nearest hotel: Posada Los Sabinos
kapa'a - nearest hotel: Pono Kai Resort
la punta - nearest hotel: La Posada de Salim
ridgecrest - nearest hotel: Best Western China Lake Inn
ganye - nearest hotel: No hotel found
iquique - nearest hotel: Hostal Vivar
nova xavantina - nearest hotel: Hotel Morada do Sol
port elizabeth - nearest hotel: Waterford Hotel
isla mujeres - nearest hotel: Villa La Bella
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
santa luzia - nearest hotel: Alcoboca
new norfolk - nearest hotel: Woodbridge on the Derwent
gereida - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
caleta de carquin

,City,Country,Lat,Lng,Humidity,Hotel Name
2,padangsidempuan,ID,1.3667,99.2667,98,Hotel Mega Permata
6,east london,ZA,-33.0153,27.9116,85,No hotel found
9,nueva gerona,CU,21.8833,-82.8000,59,Hotel Bamboo
12,blackmans bay,AU,-43.0167,147.3167,69,Villa Howden
14,mazamari,PE,-11.3258,-74.5308,98,Hospedaje El Conquistador
...,...,...,...,...,...,...
531,tanjung selor,ID,2.8372,117.3642,98,No hotel found
534,fada n'gourma,BF,12.0622,0.3578,18,Auberge Diana
565,tangalle,LK,6.0240,80.7911,93,Coco Tangalla
578,mabaruma,GY,8.2000,-59.7833,85,Hotel Broost


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    color = "City",
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)